In [1]:
# -*- coding: utf-8 -*-
"""
Yelp Fusion API code sample.
This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.
Please refer to http://www.yelp.com/developers/v3/documentation for the API
documentation.
This program requires the Python requests library, which you can install via:
`pip install -r requirements.txt`.
Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib.request
import urllib.error
from urllib.parse import quote
import pandas as pd

#from urllib import quote
#from config import yelp_key
yelp_key='l7BK7OphKU2Q3Q5xys8hneTZF4bWbMVcY_ehOwnYwH8nTsz8-Qo9FNMsXXzGUoO1BgK-xHOLE3TRmUagmeOO2eO29VlUUj2E5Au4JLUlqgJ2DwhmgumNs-Lbq4rXXXYx'

##  https://api.yelp.com/v3/businesses/{id}/reviews

In [2]:
# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY=yelp_key

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
REVIEW_PATH = '/v3/businesses/'


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 50
print(yelp_key)

l7BK7OphKU2Q3Q5xys8hneTZF4bWbMVcY_ehOwnYwH8nTsz8-Qo9FNMsXXzGUoO1BgK-xHOLE3TRmUagmeOO2eO29VlUUj2E5Au4JLUlqgJ2DwhmgumNs-Lbq4rXXXYx


In [3]:
def request(host, path, api_key, url_params=None):
    #"""Given your API_KEY, send a GET request to the API.
    #Args:
    #    host (str): The domain host of the API.
    #    path (str): The path of the API after the domain.
    #    API_KEY (str): Your API Key.
    #    url_params (dict): An optional set of query parameters in the request.
    #Returns:
    #    dict: The JSON response from the request.
    #Raises:
    #    HTTPError: An error occurs from the HTTP request.
    #"""
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [4]:
def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [5]:
def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)

In [6]:
def get_review (api_key, business_id):
    review_path = BUSINESS_PATH + business_id + "/reviews"
    return request (API_HOST, review_path, api_key)

In [7]:
def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

In [8]:
# def main():
#     parser = argparse.ArgumentParser()

#     parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
#                         type=str, help='Search term (default: %(default)s)')
#     parser.add_argument('-l', '--location', dest='location',
#                         default=DEFAULT_LOCATION, type=str,
#                         help='Search location (default: %(default)s)')

#     input_values = parser.parse_args()

#     try:
#         query_api(input_values.term, input_values.location)
#     except HTTPError as error:
#         sys.exit(
#             'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
#                 error.code,
#                 error.url,
#                 error.read(),
#             )
#         )


# if __name__ == '__main__':
#     main()

In [9]:
df1 = search(API_KEY,"Dinner","Saint Louis")



Querying https://api.yelp.com/v3/businesses/search ...


In [10]:
print(json.dumps(df1, indent=2, sort_keys=True))

{
  "businesses": [
    {
      "alias": "retreat-gastropub-saint-louis",
      "categories": [
        {
          "alias": "cocktailbars",
          "title": "Cocktail Bars"
        },
        {
          "alias": "newamerican",
          "title": "American (New)"
        },
        {
          "alias": "gastropubs",
          "title": "Gastropubs"
        }
      ],
      "coordinates": {
        "latitude": 38.637185,
        "longitude": -90.246496
      },
      "display_phone": "(314) 261-4497",
      "distance": 7062.659203982825,
      "id": "oHvEgLH6pAkcrPmeR1l3UQ",
      "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/DRdYsW4mzQLxbYnGIg-UAw/o.jpg",
      "is_closed": false,
      "location": {
        "address1": "2 N Sarah St",
        "address2": null,
        "address3": "",
        "city": "Saint Louis",
        "country": "US",
        "display_address": [
          "2 N Sarah St",
          "Saint Louis, MO 63108"
        ],
        "state": "MO",
        "zip_co

In [20]:
for business in (df1["businesses"]):
    print(business["id"])
    print(business["name"])
    print(business["review_count"])
    

oHvEgLH6pAkcrPmeR1l3UQ
Retreat Gastropub
585
aI3B47Y_AUYFP5K5JAYrlA
Polite Society
514
_4bhVmlvT9dvqydQG2TOMA
Olive + Oak
580
0AfjyySl4Rivh-QhWfrClA
58hundred
116
ZwRM2qsGJT_fkQx_MDZF8g
Akar
40
cQIh4YJlVtZI9TLF5_smOg
The Shaved Duck
1638
YT1oM3AlNSCjzKvbH8MPWA
Juniper
338
SFN5g5eexukcaZkzoiYQYg
Kounter Kulture
264
4r3Ck65DCG1T6gpWodPyrg
Bridge Tap House and Wine Bar
697
A4-2jEraYJ2w2grn_uXlHw
Blood & Sand
289
El_2Cxuq4hxqrmlg_5nzzA
Brasserie by Niche
577
wMyHVb54Ne9SNasa4kjOCg
Carnivore
130
YpGPZ-Zg89BsFUWwCXoaBw
Yellowbelly
229
b6bmnM7BSGUF1Rgxi9nb1Q
Billie Jean
61
R8t9g5nvi7VFyS8zsgmj8Q
Salt + Smoke
1431
x8AQIEjSqsW0z4vRyBetbw
The BAO
152
jQBPO3rYkNwIaOdQS5ktgQ
The Fountain On Locust
895
iRIHK8-EwpeffwvoO4nzIA
Broadway Oyster Bar
1737
S07dMpp8f9x7etbGW1L0QA
Southern
542
gBGD0dNvfLlQOAaZzCU1NQ
Twisted Tree
324
L-VNs3YquPGKVsXl2Ze-Yg
Peacemaker Lobster and Crab
669
_xWm_EgCq4dGTjo_DmEVWw
Indo
95
z7em5co2qckbAXoDGXynsA
The Blue Duck
185
GeLwhGlX8XsIsH-o3Wdc5w
360 St. Louis
615
TR8sAbY5s

In [12]:
len(df1["businesses"])

50

In [15]:
## Defining Data Frame
## id
## rating
## user id
## name
## profile id
## profile url
yelpuser = pd.DataFrame(columns = ['id','rating','user id','name','profile id','profile url'])

In [17]:
for business in (df1["businesses"]):
    #print(business["name"])
    dfb = get_review(API_KEY,business["id"])
    #print(json.dumps(dfb, indent=2, sort_keys=True))
    print(dfb["reviews"][0]["id"])
    print(dfb["reviews"][0]["rating"])
    print(dfb["reviews"][0]["id"])

Querying https://api.yelp.com/v3/businesses/oHvEgLH6pAkcrPmeR1l3UQ/reviews ...
d2SYQ7mkZUkf8WhOR0tFoQ
5
d2SYQ7mkZUkf8WhOR0tFoQ
Querying https://api.yelp.com/v3/businesses/aI3B47Y_AUYFP5K5JAYrlA/reviews ...
beAoEt2KkYtcYGVQz5xzFg
5
beAoEt2KkYtcYGVQz5xzFg
Querying https://api.yelp.com/v3/businesses/_4bhVmlvT9dvqydQG2TOMA/reviews ...
-gXw-ASVxPIcvP1qN5oSEA
5
-gXw-ASVxPIcvP1qN5oSEA
Querying https://api.yelp.com/v3/businesses/0AfjyySl4Rivh-QhWfrClA/reviews ...
mioy1QLgL0ry61jDtUyqJQ
5
mioy1QLgL0ry61jDtUyqJQ
Querying https://api.yelp.com/v3/businesses/ZwRM2qsGJT_fkQx_MDZF8g/reviews ...
0q3FlE7-XGxm-Noo4P0SeA
5
0q3FlE7-XGxm-Noo4P0SeA
Querying https://api.yelp.com/v3/businesses/cQIh4YJlVtZI9TLF5_smOg/reviews ...
8iLppnMlC5k0bkXOyyN7ag
5
8iLppnMlC5k0bkXOyyN7ag
Querying https://api.yelp.com/v3/businesses/YT1oM3AlNSCjzKvbH8MPWA/reviews ...
93AEM2RJ8eLn8T7R6Xc1iQ
5
93AEM2RJ8eLn8T7R6Xc1iQ
Querying https://api.yelp.com/v3/businesses/SFN5g5eexukcaZkzoiYQYg/reviews ...
pU2X9W-89YTpLn9pwOO5ig
5
pU2X9W-